# Retriever - Experimento

O objetivo deste componente é elencar a probabilidade de um conjunto de contextos conter a resposta a uma dada pergunta.

Este componente utiliza diferentes modelos de similaridade entre textos para a sua inferência.<br>

A tabela de dados de entrada deve possuir uma coluna de contextos, em que cada linha representa um contexto diferente, e uma coluna de perguntas em que cada linha representa uma pergunta a ser utilizada. Note que para cada pergunta serão utilizados todos os contextos fornecidos para realização da inferência, e portanto, podem haver bem mais contextos do que perguntas.

Obs: Este componente utiliza recursos da internet, portanto é importante estar conectado à rede para que este componente funcione corretamente.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [1]:
dataset = "/tmp/data/simple_q&a-18.csv" #@param {type:"string"}
context_column_name = "text" #@param {type:"string", label:"Coluna dos contextos", description:"Esta coluna será utilizada para ler os contextos e aplicar o Retriever."}
question_column_name = "question" #@param {type:"string", label:"Coluna das perguntas", description:"Esta coluna será utilizada para identificar as perguntas a serem aplicadas aos contextos."}
topn = 2 #@param {type:"number",label:"Top N contextos",description:"O número máximo de contextos para elencar por pergunta. Os contextos elencados correspondem aos que obtiveram melhor pontuação do Retriever."}
similarity_model = "paraphrase_multilingual" #@param ["paraphrase_multilingual","bm25","tf_idf","word2vec"] {type:"string",multiple:false,label:"Modelo do Retriever",description:"Modelo para a inferência do Retriever."}
proba_column_name = "retriever_score" #@param {type:"string", label:"Coluna da pontuação das respostas", description:"Esta coluna será utilizada para salvar a pontuação dos N contextos selecionados de cada pergunta."}
expected_column_name = "" #@param {type:"string", label:"Coluna de avaliação", description:"Esta coluna será utilizada para avaliar as respostas produzidas por cada pergunta, deve conter a indicação do contexto esperado para o Retriever. É uma coluna opcional, mas caso preenchido deve ser também preenchida uma coluna de identificação dos contextos."}
identifier_column_name = "" #@param {type:"string", label:"Coluna de identidicação dos Contextos", description:"Esta coluna será utilizada para identificar os contextos caso seja informado uma coluna de avaliação."}
device = "cpu" #@param ["cuda","cpu"] {type:"string",multiple:false,label:"Dispositivo",description:"Tipo de dispositivo para utilizar o componente."}

## Leitura do conjunto de dados

O exemplo abaixo faz a leitura de dados tabulares (ex: .csv).<br>
Modifique o código de acordo com o tipo de dado que desejar ler.

In [2]:
import pandas as pd

df = pd.read_csv(dataset)
df

,index,text,question,expected_answer,expected_retriever_answer
0,1,A bola é um objeto utilizado para lazer de uma...,Qual o formato da bola?,esférica,1
1,2,Golfe é um esporte no qual os jogadores usam d...,Qual o formato da bola?,esférica,1
2,3,"Jogo por tacadas[1] (stroke play[2]), também c...",Qual o formato da bola?,esférica,1
3,1,A bola é um objeto utilizado para lazer de uma...,Aonde se pratica golfe?,campo de golfe,2
4,2,Golfe é um esporte no qual os jogadores usam d...,Aonde se pratica golfe?,campo de golfe,2
5,3,"Jogo por tacadas[1] (stroke play[2]), também c...",Aonde se pratica golfe?,campo de golfe,2


## Conteúdo da tarefa

In [3]:
# Assert parameters

assert context_column_name in df.columns, f"Coluna '{context_column_name}' deve ser uma coluna do dataset."
assert question_column_name in df.columns, f"Coluna '{question_column_name}' deve ser uma coluna do dataset."
assert expected_column_name == "" or expected_column_name in df.columns, f"Coluna '{expected_column_name}' deve ser uma coluna do dataset ou deixar ou campo em branco."
assert identifier_column_name == "" or identifier_column_name in df.columns, f"Coluna '{identifier_column_name}' deve ser uma coluna do dataset ou deixar ou campo em branco."
if expected_column_name != "" and identifier_column_name == "":
    assert False, f"Para utilizar a Coluna de Avaliação é necessário preencher a Coluna de Identificação."
assert proba_column_name not in df.columns and proba_column_name != "", f"Coluna '{proba_column_name}' não pode ser vazia e nem uma coluna do dataset."
assert device in ['cuda', 'cpu'], f"Dispositivo '{device}' deve ser ou 'cuda' ou 'cpu'."
assert topn > 0, "Top N Contextos deve ser maior do que 0"
assert similarity_model in ["paraphrase_multilingual","bm25","tf_idf","word2vec"], f"Modelo '{similarity_model}' deve ser ou 'paraphrase_multilingual', ou 'bm25', ou 'tf_idf', ou 'word2vec'."

In [4]:
import torch

# Model parameters
model_parameters = {
    'device':torch.device(device) if torch.cuda.is_available() else torch.device('cpu'),
}
    
# Inference parameters
inference_parameters = {
    'context_column_name':context_column_name,
    'question_column_name':question_column_name,
    'expected_column_name':expected_column_name,
    'identifier_column_name':identifier_column_name,
    'input_columns': list(df.columns),
    'proba_column_name': proba_column_name,
    'output_columns': list(df.columns) + [proba_column_name],
    'topn': topn,
    'similarity_model': similarity_model,
}

In [5]:
!python -m spacy download pt_core_news_lg --quiet

2021-10-21 15:34:10.900176: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [6]:
from retriever import Retriever, MODEL_MAPPING

# Initilizate Retriever
sim_model = MODEL_MAPPING[inference_parameters['similarity_model']](**model_parameters)
retriever = Retriever(similarity_model=sim_model, **model_parameters)

In [7]:
# Get unique questions
unique_questions = list(df[inference_parameters['question_column_name']].unique())

# Initialize output df
output_df = pd.DataFrame(columns=inference_parameters['output_columns'])

# Iterate over each unique question
for question in unique_questions:
    
    # Filter df per question
    question_df = df[df[inference_parameters['question_column_name']] == question]
    
    # Get Contexts
    contexts = list(question_df[inference_parameters['context_column_name']])
    
    # Get indices
    context_ids = list(question_df.index)
    
    # Retrieve contexts
    topn_ids, topn_scores = retriever.retrieve_top(contexts, question, topn=inference_parameters['topn'], context_ids=context_ids)
    
    # Retrieved df
    retrieved_df = df.loc[topn_ids]
    retrieved_df.at[topn_ids, inference_parameters['proba_column_name']] = topn_scores
    
    # Save to output
    output_df = pd.concat((output_df, retrieved_df)).reset_index(drop=True)

In [8]:
output_df.to_csv(dataset, index=False)
del retriever

In [9]:
from platiagro.plotting import plot_data_table
from mrr import MRR
# Evaluate translation (Optional)

# Iff is not empty
if inference_parameters['expected_column_name'] != "":
    
    # Initializate calculators
    ranker1 = MRR(max_rank=1)
    ranker10 = MRR(max_rank=10)
    ranker100 = MRR(max_rank=100)
    
    # Get expected/predicted answers
    expected_answers = []
    predicted_answers = []
    for question in unique_questions:
        
        # Filter output data
        question_df = output_df[output_df[inference_parameters['question_column_name']] == question]
        
        # Retrieve expected answer
        expected_answer = question_df[inference_parameters['expected_column_name']].values[0]
        
        # Retrieve predicted order
        predicted_answer = question_df[inference_parameters['identifier_column_name']].values
        
        # Append answers
        expected_answers.append(expected_answer)
        predicted_answers.append(predicted_answer)
    
    # Perform calculations
    scores = {}
    scores['MRR@1'] = ranker1(predicted_answers, expected_answers)
    scores['MRR@10'] = ranker10(predicted_answers, expected_answers)
    scores['MRR@100'] = ranker100(predicted_answers, expected_answers)
    
    # Plot metrics
    plot_data_table(pd.DataFrame(scores, index = ['Valor']))

## Salva resultados da tarefa

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.<br>
Use essa pasta para salvar modelos, metadados e outros resultados.

In [10]:
from joblib import dump

artifacts = {
    "model_parameters": model_parameters,
    "inference_parameters": inference_parameters
}

dump(artifacts, "/tmp/data/retriever.joblib")

['/tmp/data/retriever.joblib']